In [1]:
from sklearn.preprocessing import Normalizer
import pandas
import numpy
import os

url = "data/abalone.data"
dataframe = pandas.read_csv(url, header=-1)

dumm = pandas.get_dummies(dataframe, prefix=None, prefix_sep='_', dummy_na=False, columns=[0,8], sparse=False, drop_first=False)

y = dumm.as_matrix(columns=dumm.columns[-28:])
# y = dumm.as_matrix(columns=dumm.columns[-3:])
X = dumm.as_matrix(columns=dumm.columns[:-28])
#  = dumm.as_matrix(columns=)
y.T.shape

(28, 4177)

In [2]:
%matplotlib inline
import math
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from sklearn.datasets.samples_generator import make_moons
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss


# # DATA FOR CLASSIFICATION
# np.random.seed(0)
# # X, y = make_blobs(n_samples=400, centers=2, n_features=2, cluster_std=0.1,random_state=0)
# X, y = make_moons(n_samples=400, shuffle=True, noise=None,random_state=0)
# plt.scatter(X[:,0], X[:,1], s=40, c=y, cmap=plt.cm.Spectral)

# reshaped_y = np.array(y).reshape(len(y),1)

# enc = OneHotEncoder()
# enc.fit(reshaped_y)
# encoded_y = enc.transform(reshaped_y).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print(y_train.shape)

(2798, 28)


In [3]:
# FUNCTIONS FOR FORWARD PASS
def ReLU(x):
    return x*(x > 0)


def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


def differentiate_ReLU(x):
    x[x <= 0] = 0
    x[x > 0] = 1
    return x

def differentiate_tanh(a_1):
    return 1 - np.tanh(a_1)**2

In [4]:
class NeuralNetwork:
    def __init__(self, neurons, x, y):
        np.random.seed(2)
        
        self.W_1 = np.random.rand(neurons,x.shape[1])
        self.W_15 = np.random.rand(neurons,neurons)
        self.W_2 = np.random.rand(neurons,neurons)
        self.W_25 = np.random.rand(neurons,neurons)
        self.W_out = np.random.rand(y.shape[1],neurons)
        self.W_ResNet = np.random.rand(y.shape[1],x.shape[1])
        
        self.b_1 = np.random.rand(neurons,1)
        self.b_15 = np.random.rand(neurons,1)
        self.b_2 = np.random.rand(neurons,1)
        self.b_25 = np.random.rand(neurons,1)
        self.b_out = np.random.rand(y.shape[1],1)
        
        
    def train(self, epochs, batch_size, X_train, X_test, y_train, y_test):
        for iteration in range(epochs):
            for i in range(len(X_train)//batch_size):
                hs = feedforward(nn, X_train[i:i+batch_size], y_train[i:i+batch_size])
                gradient = get_gradient(nn, hs)
                update_neural_network(nn, gradient)
                
            if (iteration%(epochs//1000)==0):
                hs = feedforward(nn, X_test, y_test)
                print(log_loss(y_test, hs['z_out'].T))
                
    
        if(len(X_train)%batch_size != 0):
            k = len(X_train)%batch_size
        
            hs = feedforward(nn, X_train[k:], y_train[k:])
            gradient = get_gradient(nn, hs)
            update_neural_network(nn, gradient)
            

def feedforward(network, x, y):
        forward_path = dict()
        
        forward_path['x'] = x.T
        forward_path['y'] = y.T
        
        forward_path['a_1'] = np.add(network.W_1.dot(forward_path['x']), network.b_1)
        forward_path['z_1'] = np.tanh(forward_path['a_1'])
        
        forward_path['a_15'] = np.add(network.W_15.dot(forward_path['z_1']), network.b_15)
        forward_path['z_15'] = np.tanh(forward_path['a_15'])
        
        forward_path['a_2'] = np.add(network.W_2.dot(forward_path['z_15']), network.b_2)
        forward_path['z_2'] = ReLU(forward_path['a_2'])
        
        forward_path['a_25'] = np.add(network.W_2.dot(forward_path['z_2']), network.b_25)
        forward_path['z_25'] = ReLU(forward_path['a_25'])
        
        forward_path['a_out_1'] = network.W_out.dot(forward_path['z_25'])
        forward_path['a_out_2'] = network.W_ResNet.dot(forward_path['x'])
        
        forward_path['a_out'] = np.add(np.add(forward_path['a_out_1'], forward_path['a_out_2']), network.b_out)
        forward_path['z_out'] = softmax(forward_path['a_out'])
        
        return forward_path
        
        
def get_gradient(network, hidden_state):
        gradient = dict()
        
        derivative_of_tanh = differentiate_tanh(hidden_state['a_1'])
        derivative_of_ReLU = differentiate_ReLU(hidden_state['a_2'])
 
        # application of the chain rule to find derivative of the loss function with respect to weights2 and weights1
        delta_out = hidden_state['y'] - hidden_state['z_out'] 

        
        
        delta_25 = np.multiply(derivative_of_ReLU, network.W_out.T.dot(delta_out))
        delta_2 = np.multiply(derivative_of_ReLU, network.W_25.T.dot(delta_25))
        delta_15 = np.multiply(derivative_of_tanh, network.W_2.T.dot(delta_2))
        delta_1 = np.multiply(derivative_of_tanh, network.W_15.T.dot(delta_15))
            
        dW_ResNet = delta_out.dot(hidden_state['x'].T)
        dW_out = delta_out.dot(hidden_state['z_25'].T)
        dW_25 = delta_2.dot(hidden_state['z_2'].T)
        dW_2 = delta_2.dot(hidden_state['z_15'].T)
        dW_15 = delta_15.dot(hidden_state['z_1'].T)
        dW_1 = delta_1.dot(hidden_state['x'].T)
        
        gradient['W_1'] = dW_1
        gradient['W_15'] = dW_15
        gradient['W_2'] = dW_2
        gradient['W_25'] = dW_25
        gradient['W_out'] = dW_out
        gradient['W_ResNet'] = dW_ResNet
        
        gradient['b_1'] = np.expand_dims(np.mean(delta_1, axis=1), axis=1)
        gradient['b_15'] = np.expand_dims(np.mean(delta_15, axis=1), axis=1)
        gradient['b_2'] = np.expand_dims(np.mean(delta_2, axis=1), axis=1)
        gradient['b_25'] = np.expand_dims(np.mean(delta_25, axis=1), axis=1)
        gradient['b_out'] = np.expand_dims(np.mean(delta_out, axis=1), axis=1)
        
        return gradient
    
def update_neural_network(network, gradient, learning_rate = 0.032):
    network.W_1 += learning_rate * gradient['W_1']
    network.W_15 += learning_rate * gradient['W_15']
    network.W_2 += learning_rate * gradient['W_2']
    network.W_out += learning_rate * gradient['W_out']
    network.W_ResNet += learning_rate * gradient['W_ResNet']
        
    network.b_1 += learning_rate * gradient['b_1']
    network.b_2 += learning_rate * gradient['b_2']
    network.b_out += learning_rate * gradient['b_out']

In [6]:
nn = NeuralNetwork(65, X_train, y_train)
nn.train(epochs = 20000,batch_size = 1,X_train = X_train, X_test = X_test, y_train = y_train, y_test = y_test)

10.61239035668714
2.6727813300805647
2.3138763892361154
2.2882216751274576
2.2710431592924794
2.258570841079809
2.2491334350814705
2.2417981850717097
2.2359911087896953
2.2313320768777176
2.227554797960077
2.224464823157208
2.2219158743051586
2.2197955974310624
2.218016465608499


KeyboardInterrupt: 